## Importo Librerias



In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import plotly.express as px

##Revision Gral del Data set

In [12]:
df = pd.read_csv('Data/Edad-Talla-Peso-TA.csv', sep= ';', decimal=',')
df.info()
#Aca puedo observar que a altura como a sexo tienen datos vacios

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Altura  4901 non-null   float64
 1   Peso    5000 non-null   int64  
 2   Sexo    4910 non-null   object 
 3   Edad    5000 non-null   int64  
 4   PA_max  5000 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 195.4+ KB


In [13]:
#Quiero saber el total de datos vacios que tiene el DF
df.isnull().sum()

Altura    99
Peso       0
Sexo      90
Edad       0
PA_max     0
dtype: int64

In [14]:
#Ahora quiero ver como estan los datos 
df.describe()
#Ahora puedo ver que Altura y peso tienen datos irreales y que deberia verlos

,Altura,Peso,Edad,PA_max
count,4901.000000,5000.00000,5000.000000,5000.000000
mean,8.013291,1102.30400,47.875000,16.422520
std,32.148415,8753.19627,17.300833,13.776803
min,1.450000,0.00000,18.000000,8.400000
25%,1.600000,64.00000,33.000000,12.500000
50%,1.700000,80.00000,48.000000,14.400000
75%,1.820000,104.00000,63.000000,16.900000
max,226.000000,123000.00000,77.000000,193.000000


## Buscando Outliers 

### Peso
El peso estaba lleno de datos que tuve que modificar, desde muchos 0 que tuve que transformar a Nan hasta datos que estaban en otra unidad de medida y tuve que ponerlos a todos en la misma unidad.

In [15]:
#Trato de ver si hay algun Outlier en peso
Outlierpeso = df.where((df['Peso'] > 300) | (df['Peso'] <= 0 ))#Descubri que hay muchas personas que tienen 0 como el peso en vez de NaN
Outlierpeso.sort_values('Peso', ascending=False).head(80)#tambien veo que hay diferentes unidades de medidas
#Descubri que hay bastantes outliers por encima

,Altura,Peso,Sexo,Edad,PA_max
4937,2.02,123000.0,M,34.0,13.8
4532,1.92,118000.0,F,76.0,22.1
4263,188.00,113000.0,NaN,32.0,12.2
1939,NaN,111000.0,F,30.0,23.1
1784,1.92,111000.0,F,57.0,17.8
...,...,...,...,...,...
1387,1.64,0.0,F,72.0,28.6
1104,1.78,0.0,M,30.0,10.6
4625,1.82,0.0,M,73.0,15.2
768,1.84,0.0,M,65.0,13.6


In [16]:
df['Peso'].isna().sum()#Esta operacion la hago para verificiar lo que estoy por hacer despues

0

In [17]:
df['Peso'] = df['Peso'].replace(0,np.nan)#tranforme todos los valores de 0 a NaN para que no afecten a los graficos
df['Peso'].isna().sum()

33

In [18]:
#busco encontrar los outliers de peso 
fig_Peso = px.violin(df, y="Peso",x='Sexo', box=True,points='all')
fig_Peso.show()

In [19]:
df['Peso'].loc[df.Peso>300] = df['Peso']/1000 #Modificio los datos que estan en 'g' y los paso a 'kg'

C:\Users\gino_\AppData\Local\Temp\ipykernel_24604\217603936.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Altura  4901 non-null   float64
 1   Peso    4967 non-null   float64
 2   Sexo    4910 non-null   object 
 3   Edad    5000 non-null   int64  
 4   PA_max  5000 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 195.4+ KB


### Edad

In [21]:
Edad_Df = df['Edad']
Edad_Df.where((df["Edad"] > 100) | (df['Edad'] <= 0)).sum()# por lo visto edad no tiene ningun outlier 

0.0

In [22]:
#Utilizo plotly para verificar 
fig_Edad = px.violin(df, y="Edad",x='Sexo', box=True,points='all',)
fig_Edad.show()

### Presion Maxima

In [23]:
df.sort_values(by='PA_max',ascending=False) # busco donde esta el primer valor que se encuentra en diferentes unidades de medida

,Altura,Peso,Sexo,Edad,PA_max
4988,1.70,85.0,F,63,193.0
1928,1.92,114.0,F,58,191.5
673,1.90,186.0,F,57,176.3
2269,1.64,80.0,F,70,173.7
2148,1.68,73.0,M,76,166.4
...,...,...,...,...,...
3265,1.71,83.0,F,21,8.7
1054,1.65,53.0,F,24,8.6
4068,1.76,62.0,F,21,8.6
1487,1.69,57.0,F,20,8.5


In [24]:
df['PA_max'].loc[df['PA_max']>80] = df['PA_max']/10 # Encontre los que la unidad de medida de mm esta arriba de 80 e hice que todos esos valores vayan a la unidad de cm
df.sort_values(by='PA_max',ascending=False) # lo uso para verificar

C:\Users\gino_\AppData\Local\Temp\ipykernel_24604\2178134310.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Altura,Peso,Sexo,Edad,PA_max
209,1.67,79.0,M,75,32.3
1044,1.76,88.0,F,72,31.5
3885,1.70,89.0,F,65,31.3
1296,1.92,119.0,M,64,30.6
4888,1.99,124.0,F,74,30.3
...,...,...,...,...,...
3265,1.71,83.0,F,21,8.7
1054,1.65,53.0,F,24,8.6
4068,1.76,62.0,F,21,8.6
1487,1.69,57.0,F,20,8.5


In [25]:
fig_PA = px.violin(df, y="PA_max",x='Sexo', box=True,points='all',)
fig_PA.show()

### Altura
Transforme las alturas de CM a Metros

In [26]:
df.sort_values(by='Altura',ascending=False)

,Altura,Peso,Sexo,Edad,PA_max
4642,226.0,160.0,F,33,14.1
3014,214.0,230.0,F,36,10.6
2194,207.0,121.0,F,50,13.8
642,205.0,131.0,M,58,19.7
794,203.0,117.0,F,62,18.5
...,...,...,...,...,...
4520,NaN,124.0,F,50,17.4
4538,NaN,164.0,M,70,18.2
4789,NaN,85.0,M,66,18.1
4859,NaN,80.0,M,24,13.5


In [27]:
df[df['Altura']==0] #al parecer no hay datos con 0

,Altura,Peso,Sexo,Edad,PA_max


In [28]:
df['Altura'].loc[df['Altura']> 100]  = df['Altura'] /100 #Modifico todos los valores que estan en centimetros y los paso a metros

C:\Users\gino_\AppData\Local\Temp\ipykernel_24604\2517712695.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
fig_Alt = px.violin(df, y="Altura",x='Sexo', box=True,points='all',)
fig_Alt.show()

### Sexo
Modifique las variables F y M por 1 y 0

In [30]:
df['Sexo'] = np.where(df['Sexo']=='F',1,0) # transforme la variables F y M en variables numericas para poder trabajar mejor con ellas
df

,Altura,Peso,Sexo,Edad,PA_max
0,1.47,53.0,0,33,12.3
1,1.72,90.0,1,28,13.3
2,1.60,118.0,0,77,24.9
3,1.76,97.0,0,52,18.3
4,1.81,118.0,1,73,25.8
...,...,...,...,...,...
4995,1.85,105.0,0,74,13.6
4996,1.78,66.0,1,31,9.9
4997,1.81,168.0,1,57,15.5
4998,2.04,99.0,0,44,11.8


### IMC

In [31]:
df['IMC'] = df['Peso']/(df['Altura']**2)
df

,Altura,Peso,Sexo,Edad,PA_max,IMC
0,1.47,53.0,0,33,12.3,24.526818
1,1.72,90.0,1,28,13.3,30.421850
2,1.60,118.0,0,77,24.9,46.093750
3,1.76,97.0,0,52,18.3,31.314566
4,1.81,118.0,1,73,25.8,36.018437
...,...,...,...,...,...,...
4995,1.85,105.0,0,74,13.6,30.679328
4996,1.78,66.0,1,31,9.9,20.830703
4997,1.81,168.0,1,57,15.5,51.280486
4998,2.04,99.0,0,44,11.8,23.788927


## Graficas de Analisis

In [32]:
df

,Altura,Peso,Sexo,Edad,PA_max,IMC
0,1.47,53.0,0,33,12.3,24.526818
1,1.72,90.0,1,28,13.3,30.421850
2,1.60,118.0,0,77,24.9,46.093750
3,1.76,97.0,0,52,18.3,31.314566
4,1.81,118.0,1,73,25.8,36.018437
...,...,...,...,...,...,...
4995,1.85,105.0,0,74,13.6,30.679328
4996,1.78,66.0,1,31,9.9,20.830703
4997,1.81,168.0,1,57,15.5,51.280486
4998,2.04,99.0,0,44,11.8,23.788927


1.   Alta realacion entre la altura y el peso.
2.   Alta relacion entre Presion maxima y la edad
3.   Alta relacion entre la Altura y el IMC (tiene logica se calcula con la altura)

In [33]:
fig = px.imshow(df.corr(),title='Correalcion de datos',text_auto=True,aspect="auto")
fig.show()


In [34]:
fig = px.scatter_3d(df, x='IMC', y='Edad', z='PA_max', color='Sexo',title='Grafica de puntos sobre el IMC, la edad y la presion Maxima')
fig.show()

In [35]:
fig_Edad_PA = px.scatter(df, x='Edad', y='PA_max',trendline='ols', trendline_color_override='Blue', title='analisis de la relacion entre la edad y la presion')
fig_Edad_PA.show()

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
fig_Peso_PA= px.scatter(df, x='Peso', y='PA_max',trendline='ols', trendline_color_override='Blue',title='analisis de la relacion entre el peso y la presion')
fig_Peso_PA.show()

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
fig_Altura_PA = px.scatter(df, x='Altura', y='PA_max',trendline='ols', trendline_color_override='Blue', title='analisis de la relacion entre la altura y la presion')
fig_Altura_PA.show()

ModuleNotFoundError: No module named 'statsmodels'

In [37]:
fig_Edad_IMC = px.scatter(df, x='Edad', y='IMC',trendline='ols', trendline_color_override='Blue',title='analisis de la relacion entre la edad y el IMC')
fig_Edad_IMC.show()

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
fig = px.violin(df, y="PA_max", color="Sexo", violinmode='overlay')
fig.show()

In [36]:
fig_IMC = px.violin(df, y="IMC", color="Sexo", violinmode='overlay')
fig_IMC.show()